In [1]:
import pandas as pd

In [52]:
nba = pd.read_csv('2017-18_teamBoxScore.csv')
nba.head()

,PrimKey,opptKey,gmDate,teamAbbr,teamLoc,teamRslt,teamMin,teamDayOff,teamPTS,teamAST,...,opptFIC40,opptOrtg,opptDrtg,opptEDiff,opptPlay%,opptAR,opptAST/TO,opptSTL/TO,poss,pace
0,43025BOS,43025CLE,10/17/2017,BOS,Away,Loss,241,0,99,24,...,55.5208,101.7143,98.7227,2.9916,0.4176,14.6154,1.1176,17.6471,100.2809,99.8648
1,43025CLE,43025BOS,10/17/2017,CLE,Home,Win,240,0,102,19,...,62.5519,98.7227,101.7143,-2.9916,0.3956,17.7778,2.0000,91.6667,100.2809,100.2809
2,43025HOU,43025GS,10/17/2017,HOU,Away,Win,239,0,122,28,...,86.2033,118.0353,119.0108,-0.9755,0.4725,24.2442,2.0000,29.4118,102.5117,102.9406
3,43025GS,43025HOU,10/17/2017,GS,Home,Loss,241,0,121,34,...,81.9038,119.0108,118.0353,0.9755,0.4700,19.1309,2.1538,69.2308,102.5117,102.0863
4,43026CHA,43026DET,10/18/2017,CHA,Away,Loss,238,0,90,16,...,70.7113,103.0506,90.9270,12.1236,0.4271,17.8731,2.6667,155.5556,98.9805,99.8123


In [9]:
#example of how rolling mean works
pd.rolling_mean(nba['teamPTS'][nba['teamAbbr']=='BOS'].shift(),window=6, min_periods = 6).head(10)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=6,min_periods=6,center=False).mean()
  """Entry point for launching an IPython kernel.


0             NaN
13            NaN
38            NaN
101           NaN
132           NaN
158           NaN
183    100.500000
217    102.000000
256    104.166667
274    104.000000
Name: teamPTS, dtype: float64

In [53]:
#select columns that are numeric for trailing calc
nba_numbers = nba.select_dtypes(exclude='object')

In [54]:
#merge PrimKey back into numeric df
a = pd.DataFrame(nba['PrimKey'])
nba_numbers_1 = nba_numbers.merge(a,left_index=True, right_index=True)
nba_numbers_1.head()

,teamMin,teamDayOff,teamPTS,teamAST,teamTO,teamSTL,teamBLK,teamPF,teamFGA,teamFGM,...,opptOrtg,opptDrtg,opptEDiff,opptPlay%,opptAR,opptAST/TO,opptSTL/TO,poss,pace,PrimKey
0,241,0,99,24,12,11,4,24,88,36,...,101.7143,98.7227,2.9916,0.4176,14.6154,1.1176,17.6471,100.2809,99.8648,43025BOS
1,240,0,102,19,17,3,4,25,83,38,...,98.7227,101.7143,-2.9916,0.3956,17.7778,2.0000,91.6667,100.2809,100.2809,43025CLE
2,239,0,122,28,13,9,5,16,97,47,...,118.0353,119.0108,-0.9755,0.4725,24.2442,2.0000,29.4118,102.5117,102.9406,43025HOU
3,241,0,121,34,17,5,9,25,80,43,...,119.0108,118.0353,0.9755,0.4700,19.1309,2.1538,69.2308,102.5117,102.0863,43025GS
4,238,0,90,16,17,4,3,15,73,29,...,103.0506,90.9270,12.1236,0.4271,17.8731,2.6667,155.5556,98.9805,99.8123,43026CHA


In [55]:
#get column names into list
num_fields = nba_numbers_1.columns.tolist()

In [56]:
#exclude fields that don't make sense to do a trailing average of
exclude = ['teamDayOff']
team_fields = [x for x in num_fields if x not in exclude]
#exclude opponent stats as it is redudant 
#as there is a line for the same game for the opponent too and 
#will merge that back later
team_fields = [x for x in team_fields if 'oppt' not in x]

In [57]:
#remove PrimKey
trailing_fields = [x for x in team_fields if x != 'PrimKey']

In [58]:
#get team names for loop below
teams = nba['teamAbbr'].unique()
len(teams) #correct should be 30

30

In [59]:
counter = 0
for i in teams:
    temp = nba[team_fields][nba['teamAbbr']==i]
    for x in trailing_fields:
        temp[x+'_T6'] = pd.rolling_mean(temp[x].shift(),window=6, min_periods = 6)
    if counter == 0:
        newDF = temp
    else:
        newDF = newDF.append(temp)
        
    counter = 1

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=6,min_periods=6,center=False).mean()
  """


In [60]:
len(team_fields) + len(trailing_fields) # number of fields for newDF

89

In [61]:
print(newDF.shape)
print(nba.shape)

(2460, 89)
(2460, 97)


In [73]:
#create final data set with trailing averages for team and relevant fields prior like:
#home/away, days off, and the dependent variable margin of victory
nba['MarginOfVictory'] = nba['teamPTS'] - nba['opptPTS']
nba_final = nba[['PrimKey','opptKey','MarginOfVictory','teamDayOff','teamLoc']]
nba_final.head()

,PrimKey,opptKey,MarginOfVictory,teamDayOff,teamLoc
0,43025BOS,43025CLE,-3,0,Away
1,43025CLE,43025BOS,3,0,Home
2,43025HOU,43025GS,1,0,Away
3,43025GS,43025HOU,-1,0,Home
4,43026CHA,43026DET,-12,0,Away


In [66]:
#create list with only trailing fields
fields_t6 = [x for x in newDF.columns.tolist() if 'T6' in x]
fields_t6.append('PrimKey')

In [74]:
#merge with trailing data for team
nba_final = nba_final.merge(newDF[fields_t6],on='PrimKey')

In [75]:
nba_final.shape

(2460, 49)

In [77]:
len(fields_t6) #45 plus the five fields in nba_final minus the primkey = 
#49 columns as shown abvoe

45

In [78]:
#merge in opponent trailing averages
nba_final = nba_final.merge(newDF[fields_t6], left_on='opptKey', right_on='PrimKey')

In [80]:
nba_final.tail()

,PrimKey_x,opptKey,MarginOfVictory,teamDayOff,teamLoc,teamMin_T6_x,teamPTS_T6_x,teamAST_T6_x,teamTO_T6_x,teamSTL_T6_x,...,teamOrtg_T6_y,teamDrtg_T6_y,teamEDiff_T6_y,teamPlay%_T6_y,teamAR_T6_y,teamAST/TO_T6_y,teamSTL/TO_T6_y,poss_T6_y,pace_T6_y,PrimKey_y
2455,43201LAC,43201LAL,-15,2,Home,239.833333,103.833333,21.666667,13.333333,6.333333,...,102.283817,108.375800,-6.091983,0.405317,16.500817,1.401300,43.239367,98.756250,97.161500,43201LAL
2456,43201UTA,43201POR,-9,1,Away,240.000000,115.500000,28.166667,15.333333,8.333333,...,104.276217,104.558817,-0.282600,0.418883,16.115717,1.623800,54.273550,97.100500,97.108950,43201POR
2457,43201POR,43201UTA,9,2,Home,240.000000,101.333333,21.666667,14.333333,7.666667,...,118.820267,98.601000,20.219267,0.495783,20.606517,1.875783,55.735233,97.258133,97.264333,43201UTA
2458,43201HOU,43201SAC,-13,1,Away,239.500000,101.666667,19.666667,14.333333,8.500000,...,99.838067,105.721317,-5.883250,0.411517,16.920350,1.609717,45.208333,92.861783,92.736183,43201SAC
2459,43201SAC,43201HOU,13,2,Home,240.333333,92.666667,21.666667,13.666667,5.666667,...,105.814033,105.734983,0.079050,0.406400,15.625433,1.464283,62.235683,95.989983,96.190500,43201HOU


In [83]:
home = pd.get_dummies(nba_final['teamLoc'])
home = home['Home']
home.head()

0    0
1    1
2    0
3    1
4    0
Name: Home, dtype: uint8

In [84]:
nba_final['Home'] = home

In [89]:
nba_final.drop(['teamLoc'],axis=1,inplace=True)
nba_final.tail()

,PrimKey_x,opptKey,MarginOfVictory,teamDayOff,teamMin_T6_x,teamPTS_T6_x,teamAST_T6_x,teamTO_T6_x,teamSTL_T6_x,teamBLK_T6_x,...,teamDrtg_T6_y,teamEDiff_T6_y,teamPlay%_T6_y,teamAR_T6_y,teamAST/TO_T6_y,teamSTL/TO_T6_y,poss_T6_y,pace_T6_y,PrimKey_y,Home
2455,43201LAC,43201LAL,-15,2,239.833333,103.833333,21.666667,13.333333,6.333333,3.666667,...,108.375800,-6.091983,0.405317,16.500817,1.401300,43.239367,98.756250,97.161500,43201LAL,1
2456,43201UTA,43201POR,-9,1,240.000000,115.500000,28.166667,15.333333,8.333333,5.833333,...,104.558817,-0.282600,0.418883,16.115717,1.623800,54.273550,97.100500,97.108950,43201POR,0
2457,43201POR,43201UTA,9,2,240.000000,101.333333,21.666667,14.333333,7.666667,3.833333,...,98.601000,20.219267,0.495783,20.606517,1.875783,55.735233,97.258133,97.264333,43201UTA,1
2458,43201HOU,43201SAC,-13,1,239.500000,101.666667,19.666667,14.333333,8.500000,4.833333,...,105.721317,-5.883250,0.411517,16.920350,1.609717,45.208333,92.861783,92.736183,43201SAC,0
2459,43201SAC,43201HOU,13,2,240.333333,92.666667,21.666667,13.666667,5.666667,4.166667,...,105.734983,0.079050,0.406400,15.625433,1.464283,62.235683,95.989983,96.190500,43201HOU,1


In [90]:
nba_final.to_csv('2017-18_trailing6TeamBoxScore.csv')